### 1. Crawling data from community website

In [ ]:
chromedriver = './files/chromedriver'
driver = webdriver.Chrome(chromedriver)
driver.get('http://www.ssodam.com/')

button = driver.find_element_by_css_selector("a[data-target='#loginModal']")
button.click()

wait = WebDriverWait(driver,10)

email = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[class='form-control']")))
email.clear()
email.send_keys('****')

password = driver.find_element_by_css_selector("input[placeholder='Password']")
password.clear()
password.send_keys('****')

login = driver.find_element_by_css_selector("input[class='submit btn btn-default']")
login.click()

time.sleep(1)

alert = driver.switch_to_alert()
alert.accept()

crawl = OrderedDict()
crawl['title'] = []
crawl['body'] = []
crawl['date'] = []
for i in range(213249, 223249):
    driver.get('http://www.ssodam.com/content/{}'.format(i))
    try:
        title = driver.find_element_by_xpath("//div[@class='board-title']")
        body = driver.find_element_by_xpath("//div[@class='board-content mverdana']")
        date = driver.find_element_by_xpath("//div[@class='board-date row col-lg-3 col-xs-12 desktop-hide']")
    except:
        crawl['title'].append('empty')
        crawl['body'].append('empty')
        crawl['date'].append('empty')
        continue
    
    try:
        crawl['title'].append(title.text.encode('utf-8'))
        crawl['body'].append(body.text.encode('utf-8'))
        crawl['date'].append(date.text.encode('utf-8'))
    except :
        continue

df_crawl = pd.DataFrame(crawl)
df_crawl.to_csv('crawl213249_223249.csv')

### 2. Counting words used frequently

In [ ]:
df_frozen = pd.read_csv('./files/seodam_together_notags0322.csv').drop(['Unnamed: 0'], axis=1)
df_unfrozen = pd.read_csv('./files/unfrozen2_3500.csv').drop(['Unnamed: 0'], axis=1)

tagger = Twitter()
def tokenize(doc):
    token_list = []
    for t in tagger.pos(doc, norm=True, stem=True):
        if t[1] != 'Josa' and t[1] != 'Punctuation' and t[1] != 'Determiner' and t[1] != 'URL' :
            token_list.append('/'.join(t))
    return token_list

frozen = np.array(df_frozen['text'])
frozen_str = ' '.join(frozen).decode('utf-8')
unfrozen = np.array(df_unfrozen['text2'])
unfrozen_str = ' '.join(unfrozen).decode('utf-8')

words = tokenize(frozen_str)
counter = Counter(words)
stop_words = []
for k, v in counter.most_common(100):
    stop_words.append(k)

words2 = tokenize(unfrozen_str)
counter2 = Counter(words2)
stop_words2 = []
for k, v in counter2.most_common(100):
    stop_words2.append(k)
    
pprint(stop_words)
print '*' * 50
pprint(stop_words2)

### 3. BOW encoding and fitting model

+ using nouns only

In [2]:
df_frozen = pd.read_csv('./files/seodam_together_notags0323.csv').drop(['Unnamed: 0'], axis=1)
df_unfrozen = pd.read_csv('./files/unfrozen2_3500.csv').drop(['Unnamed: 0'], axis=1)[:3213]

unfrozen = np.array(df_unfrozen['text2'])
frozen = np.array(df_frozen['text'])
seodam_x = np.append(unfrozen, frozen)
seodam_y = np.append(np.zeros(3213, dtype=int), np.ones(3213, dtype=int))

X_train, X_test, y_train, y_test = train_test_split(seodam_x, seodam_y, test_size=0.1, random_state=2)

pos_tagger = Twitter()
def tokenize(doc):
    return pos_tagger.nouns(doc)

vect = CountVectorizer(tokenizer=tokenize)
vect.fit(X_train)
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train, y_train)

result = clf.predict(X_test)

print classification_report(y_test, result)
print '*' * 50
print confusion_matrix(y_test, result)

             precision    recall  f1-score   support

          0       0.83      0.69      0.75       316
          1       0.74      0.86      0.79       327

avg / total       0.78      0.77      0.77       643

**************************************************
[[217  99]
 [ 46 281]]


+ using all tags

In [3]:
df_frozen = pd.read_csv('./files/seodam_together_notags0323.csv').drop(['Unnamed: 0'], axis=1)
df_unfrozen = pd.read_csv('./files/unfrozen2_3500.csv').drop(['Unnamed: 0'], axis=1)[:3213]

unfrozen = np.array(df_unfrozen['text2'])
frozen = np.array(df_frozen['text'])
seodam_x = np.append(unfrozen, frozen)
seodam_y = np.append(np.zeros(3213, dtype=int), np.ones(3213, dtype=int))

X_train, X_test, y_train, y_test = train_test_split(seodam_x, seodam_y, test_size=0.1, random_state=2)

pos_tagger = Twitter()
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

vect = CountVectorizer(tokenizer=tokenize)
vect.fit(X_train)
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train, y_train)

result = clf.predict(X_test)

print classification_report(y_test, result)
print '*' * 50
print confusion_matrix(y_test, result)

             precision    recall  f1-score   support

          0       0.83      0.78      0.80       316
          1       0.80      0.84      0.82       327

avg / total       0.81      0.81      0.81       643

**************************************************
[[246  70]
 [ 51 276]]


+ without 'josa', punctuations, 'determiner', 'url'

In [5]:
df_frozen = pd.read_csv('./files/seodam_together_notags0323.csv').drop(['Unnamed: 0'], axis=1)
df_unfrozen = pd.read_csv('./files/unfrozen2_3500.csv').drop(['Unnamed: 0'], axis=1)[:3213]

unfrozen = np.array(df_unfrozen['text2'])
frozen = np.array(df_frozen['text'])
seodam_x = np.append(unfrozen, frozen)
seodam_y = np.append(np.zeros(3213, dtype=int), np.ones(3213, dtype=int))

X_train, X_test, y_train, y_test = train_test_split(seodam_x, seodam_y, test_size=0.1, random_state=2)

tagger = Twitter()
def tokenize(doc):
    token_list = []
    for t in tagger.pos(doc, norm=True, stem=True):
        if t[1] != 'Josa' and t[1] != 'Punctuation' and t[1] != 'Determiner' and t[1] != 'URL' :
            token_list.append('/'.join(t))
    return token_list

vect = CountVectorizer(tokenizer=tokenize)
vect.fit(X_train)
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train, y_train)

result = clf.predict(X_test)

print classification_report(y_test, result)
print '*' * 50
print confusion_matrix(y_test, result)

             precision    recall  f1-score   support

          0       0.81      0.78      0.80       316
          1       0.80      0.83      0.81       327

avg / total       0.80      0.80      0.80       643

**************************************************
[[247  69]
 [ 57 270]]


+ using stop_words

In [7]:
df_frozen = pd.read_csv('./files/seodam_together_notags0323.csv').drop(['Unnamed: 0'], axis=1)
df_unfrozen = pd.read_csv('./files/unfrozen2_3500.csv').drop(['Unnamed: 0'], axis=1)[:3213]

unfrozen = np.array(df_unfrozen['text2'])
frozen = np.array(df_frozen['text'])
seodam_x = np.append(unfrozen, frozen)
seodam_y = np.append(np.zeros(3213, dtype=int), np.ones(3213, dtype=int))

X_train, X_test, y_train, y_test = train_test_split(seodam_x, seodam_y, test_size=0.1, random_state=2)

with open('stopwords.txt', 'r') as reader :
    stop_words0 = reader.readlines()
stop_words = stop_words0[0].split(',')

stop_words2 = []
for words in stop_words:
    stop_words2.append(words.decode('utf-8'))

tagger = Twitter()
def tokenize(doc):
    token_list = []
    for t in tagger.pos(doc, norm=True, stem=True):
        if t[1] != 'Josa' and t[1] != 'Punctuation' and t[1] != 'Determiner' and t[1] != 'URL' :
            token_list.append('/'.join(t))
    return token_list

vect = CountVectorizer(tokenizer=tokenize, stop_words=stop_words2[:15])
vect.fit(X_train)
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train, y_train)

result = clf.predict(X_test)

print classification_report(y_test, result)
print '*' * 50
print confusion_matrix(y_test, result)

             precision    recall  f1-score   support

          0       0.81      0.79      0.80       316
          1       0.80      0.82      0.81       327

avg / total       0.80      0.80      0.80       643

**************************************************
[[250  66]
 [ 60 267]]


+ using Tfidf(with stop words)

In [6]:
df_frozen = pd.read_csv('./files/seodam_together_notags0323.csv').drop(['Unnamed: 0'], axis=1)
df_unfrozen = pd.read_csv('./files/unfrozen2_3500.csv').drop(['Unnamed: 0'], axis=1)[:3213]

unfrozen = np.array(df_unfrozen['text2'])
frozen = np.array(df_frozen['text'])
seodam_x = np.append(unfrozen, frozen)
seodam_y = np.append(np.zeros(3213, dtype=int), np.ones(3213, dtype=int))

X_train, X_test, y_train, y_test = train_test_split(seodam_x, seodam_y, test_size=0.1, random_state=2)

with open('stopwords.txt', 'r') as reader :
    stop_words0 = reader.readlines()
stop_words = stop_words0[0].split(',')

stop_words2 = []
for words in stop_words:
    stop_words2.append(words.decode('utf-8'))

tagger = Twitter()
def tokenize(doc):
    token_list = []
    for t in tagger.pos(doc, norm=True, stem=True):
        if t[1] != 'Josa' and t[1] != 'Punctuation' and t[1] != 'Determiner' and t[1] != 'URL' :
            token_list.append('/'.join(t))
    return token_list

tfidv = TfidfVectorizer(tokenizer=tokenize, stop_words=stop_words2[:15])
X_train = tfidv.fit_transform(X_train)
X_test = tfidv.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train, y_train)

result = clf.predict(X_test)

print classification_report(y_test, result)
print '*' * 50
print confusion_matrix(y_test, result)

             precision    recall  f1-score   support

          0       0.78      0.86      0.82       316
          1       0.85      0.76      0.81       327

avg / total       0.82      0.81      0.81       643

**************************************************
[[273  43]
 [ 77 250]]
